# Task 1

Линейная модель:

$Y = X\beta + \varepsilon$, где 
- $\vec\varepsilon \sim \mathcal{N}(0, \text{diag}(\sigma^2))$
- $X\in M_{n \times 3}$
- $Y\in M_{n\times 1}$

Столбцы $X$ - признаки: продолжительность, танцевальность и энергичность;

$Y$ - столбец значений популярности.

Подготовим данные.

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('songs.csv')

X = df[['song_duration_ms', 'danceability', 'energy']].values
y = df['song_popularity'].values

print("X:", X.shape)
print("y:", y.shape)

X: (18835, 3)
y: (18835,)


In [16]:
# 1) simple
XtX = X.T @ X
Xty = X.T @ y
hat_beta = np.linalg.solve(XtX, Xty)

# 2) SVD
# U, s, Vt = np.linalg.svd(X, full_matrices=False)
# hat_beta = Vt.T @ np.diag(1.0 / s) @ U.T @ y

y_pred = X @ hat_beta
rss = np.sum((y - y_pred)**2) # residual error
n = X.shape[0]
m = X.shape[1]
df = n - m
hat_sigma = rss / df

print("beta: ", hat_beta)
print("sigma^2: ", hat_sigma)

beta:  [6.91437859e-05 4.53102909e+01 1.25473532e+01]
sigma^2:  524.3467426503969


Теперь найдем доверительные интервалы для $\beta$ и $\sigma^2$

Для $\sigma^2$: 

$\frac{S^2(\hat \beta)}{\sigma^2} \sim \chi^2(n-m)$, значит

$1 - \alpha = P(q_\alpha\leq \frac{S^2(\hat \beta)}{\sigma^2} \leq q_{1-\alpha}) = P(\frac{S^2(\hat \beta)}{q_\alpha}\ge \sigma^2 \ge \frac{S^2(\hat \beta)}{q_{1-\alpha}})$


In [ ]:
import scipy.stats as ss

alpha = 0.05 

chi2_lower = ss.chi2.ppf(alpha/2, df) 
chi2_upper = ss.chi2.ppf(1 - alpha/2, df)

ci_sigma_lower = rss / chi2_upper
ci_sigma_upper = rss / chi2_lower

print("[", ci_sigma_lower, ", ", ci_sigma_upper, "]", sep='')
  

[513.9149360050505, 535.1009473449335]


Для $\beta$:

$\sqrt{n-m}\frac{\hat\beta_i - \beta_i}{\sqrt{A^{-1}_{ii}S^2(\hat\beta)}} \sim T(n-m)$

$\hat\beta_i - \frac{q_\alpha \sqrt{A^{-1}_{ii}S^2(\hat\beta)}}{\sqrt{n-m}} \ge \beta_i \ge \hat\beta_i - \frac{q_{1-\alpha} \sqrt{A^{-1}_{ii}S^2(\hat\beta)}}{\sqrt{n-m}}$

In [56]:
XtX_inv = np.linalg.inv(X.T @ X)
se_beta = np.sum(y_pred.T @ y_pred)

alpha = 0.05
t_lower = ss.t.ppf(alpha/2, df)
t_upper = ss.t.ppf(1 - alpha/2, df)

ci_beta_lower = hat_beta - t_upper * np.sqrt(se_beta / df)
ci_beta_upper = hat_beta - t_lower * np.sqrt(se_beta / df)

for i in range(X.shape[1]):
  print("[", 
        hat_beta[i] - t_upper * np.sqrt(se_beta * XtX_inv[i,i] / df), 
        ", ", 
        hat_beta[i] - t_lower * np.sqrt(se_beta * XtX_inv[i,i] / df), 
        "]", sep='')

[5.905968092039562e-05, 7.92278907928656e-05]
[41.77561806687407, 48.84496381005146]
[9.294036554331747, 15.800669817886146]


Вычислим коэфиценет детерминации $R^2$

$R_m = \frac{\sum (Y_i - \overline {Y_i})(\hat{Y_i} - \overline{\hat{Y_i}})}{\sqrt{\sum(Y_i - \overline {Y_i})^2\sum (\hat{Y_i} - \overline{\hat{Y_i}})^2}}$

In [57]:
y_mean = np.mean(y)
y_pred_mean = np.mean(y_pred)

r_m = np.sum((y - y_mean) * (y_pred - y_pred_mean))
r_m /= np.sqrt(np.sum((y - y_mean) ** 2))
r_m /= np.sqrt(np.sum((y_pred - y_pred_mean) ** 2))

print("R_m: ", r_m)

R_m:  0.07828612913813421


Проверим, что есть прямая зависимость popularity(energy) с помощью рангового критерия Спирмена и критерия линейной зависимости Пирсона.

In [31]:
energy = X[:, 2]

# Spireman
rho_spearman, p_value_spearman = ss.spearmanr(energy, y)
print(f"Rho for Spireman: {rho_spearman:.4f}")
print(f"p-value: {p_value_spearman:.4f}")
print()

# Pearson
rho_pearson, p_value_pearson = ss.pearsonr(energy, y)
print(f"Rho for Pearson: {rho_pearson:.4f}")
print(f"p-value: {p_value_pearson:.4f}")


Rho for Spireman: 0.0044
p-value: 0.5420

Rho for Pearson: 0.0014
p-value: 0.8514


Для проверки зависимости, добъемся попадания в критическую область в критерии независимости хи-хвадрат

In [ ]:
x = X[:, 0]

bins = 10
x_bins = np.quantile(x, np.linspace(0, 1, bins + 1))
y_bins = np.quantile(y, np.linspace(0, 1, bins + 1))

x_disc = np.digitize(x, x_bins[1:-1])
y_disc = np.digitize(y, y_bins[1:-1])

cont_table = np.zeros((bins, bins))
for i in range(bins):
  for j in range(bins):
    cont_table[i, j] = np.sum((x_disc == i) & (y_disc == j))

_, p_value, _, _ = ss.chi2_contingency(cont_table)
print(f"{p_value}")
print("p_value < 0.05: ", p_value < 0.05)

[[189. 262. 206. 225. 277. 169. 133. 180. 108. 135.]
 [184. 211. 199. 184. 230. 183. 192. 199. 159. 142.]
 [178. 171. 188. 170. 230. 152. 211. 168. 150. 265.]
 [163. 150. 146. 152. 203. 183. 184. 244. 191. 268.]
 [181. 193. 139. 152. 230. 191. 173. 194. 154. 275.]
 [187. 156. 181. 166. 199. 154. 201. 196. 196. 249.]
 [182. 183. 175. 167. 209. 172. 168. 235. 157. 235.]
 [179. 159. 143. 185. 191. 199. 213. 221. 210. 183.]
 [183. 182. 160. 164. 199. 187. 210. 214. 213. 172.]
 [228. 221. 203. 190. 208. 194. 151. 168. 180. 141.]]
7.594898628058912e-50
p_value < 0.05:  True


Обучим модель потупее, где $X$ состоит только из 1 признака (вторые два = 0).

In [ ]:
XX = X[:, [0]].copy()
print(XX.shape)
XXtXX = XX.T @ XX
XXty = XX.T @ y
hat_beta_2 = np.linalg.solve(XXtXX, XXty)

y_pred_2 = XX @ hat_beta_2
rss_2 = np.sum((y - y_pred_2)**2)
F = (n - 3) / 2 * (rss_2 - rss) / rss
print(ss.f.ppf(1-alpha, 2, n-3))
print(F)

(18835, 1)
2.9962088752785556
1.0
